In [22]:

import DataSetQuery
import AsyncDataSetQuery
import datetime
import json
from pandas.io.json import json_normalize
import pandas as pd
import datetime
from datetime import timedelta
from dateutil.relativedelta import *

import MalardHelpers as m

environmentName = 'DEVv2'
#Sets the URL of the Malard ServiceGateway.
query = DataSetQuery.DataSetQuery('http://localhost:9000',environmentName)

asyncquery = AsyncDataSetQuery.AsyncDataSetQuery( 'ws://localhost:9000',environmentName)


In [23]:
query.getParentDataSets()

'[{"name":"cryotempo"},{"name":"jatest"},{"name":"mtngla"},{"name":"mtnglauat"},{"name":"test"}]'

In [24]:
query.getDataSets('cryotempo')

'[{"name":"AIS_BaselineC_FilteredPC","region":"antarctic"},{"name":"oib","region":"antarctic"},{"name":"oib","region":"greenland"},{"name":"GRIS_BaseC_Q2","region":"greenland"}]'

In [25]:
parentDs = 'cryotempo'
dataSet = 'AIS_BaselineC_FilteredPC'
region = 'antarctic'

bbox = json.loads(query.getDataSetBoundingBox( parentDs,dataSet,region ))

In [26]:
print(bbox)

{'gridCellMinX': -2700000, 'gridCellMaxX': 2700000, 'gridCellMinY': -2200000, 'gridCellMaxY': 2200000, 'minTime': 1291164342, 'maxTime': 1483226555, 'totalPoints': 9540724423, 'numberOfShards': 62630}


In [27]:
minX = bbox['gridCellMinX']
maxX = bbox['gridCellMaxX']
minY = bbox['gridCellMinY']
maxY = bbox['gridCellMaxY']
minT = datetime.datetime.fromtimestamp(bbox['minTime'])
maxT = datetime.datetime.fromtimestamp(bbox['maxTime'])

gridCellsDf = json_normalize(json.loads(query.getGridCells( parentDs, dataSet, region, minX, maxX, minY, maxY, minT, maxT )))


In [28]:
shardsDf = json_normalize(json.loads(query.getShards( parentDs, dataSet, region, minX, maxX, minY, maxY, minT, maxT )))

print(shardsDf.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62630 entries, 0 to 62629
Data columns (total 8 columns):
maxT              62630 non-null int64
maxX              62630 non-null float64
maxY              62630 non-null float64
minT              62630 non-null int64
minX              62630 non-null float64
minY              62630 non-null float64
numberOfPoints    62630 non-null int64
shardName         62630 non-null object
dtypes: float64(4), int64(3), object(1)
memory usage: 3.8+ MB
None


In [29]:


def checkGridCellByMonth( gc ):
    count = 0 
    gcStartTime = datetime.datetime.fromtimestamp(gc['minTime'])
    gcEndTime = datetime.datetime.fromtimestamp(gc['maxTime'])
    minX = int(gc['gridCellMinX'])
    maxX = int(gc['gridCellMaxX'])
    minY = int(gc['gridCellMinY'])
    maxY = int(gc['gridCellMaxY'])
    
    adjStartTime = datetime.datetime( gcStartTime.year, gcStartTime.month, 1, 0, 0,0  )  
    incStartTime = adjStartTime +relativedelta(months=+1)-timedelta(seconds=1)
    
    while adjStartTime < gcEndTime:
        minT = adjStartTime
        maxT = incStartTime
        
        cacheResult = asyncquery.executeQuery( parentDs, dataSet, region, minX, maxX, minY, maxY, minT, maxT,[],[] )
        
        if cacheResult.status != 'Error: Empty resultset.':
            df = m.getDataFrameFromNetCDF(cacheResult.resultFileName)
            count = count + len(df['x'])
            asyncquery.releaseCache(cacheResult.resultFileName)
        
        adjStartTime = incStartTime
        incStartTime = incStartTime +relativedelta(months=+1)-timedelta(seconds=1)
    
    return count

def checkGridCell( gc ):
    count = 0 
    gcStartTime = datetime.datetime.fromtimestamp(gc['minTime'])
    gcEndTime = datetime.datetime.fromtimestamp(gc['maxTime'])
    minX = int(gc['gridCellMinX'])
    maxX = int(gc['gridCellMaxX'])
    minY = int(gc['gridCellMinY'])
    maxY = int(gc['gridCellMaxY'])
    minT = gcStartTime
    maxT = gcEndTime
        
    cacheResult = asyncquery.executeQuery( parentDs, dataSet, region, minX, maxX, minY, maxY, minT, maxT,['x'],[] )
        
    if cacheResult.status != 'Error: Empty resultset.':
        df = m.getDataFrameFromNetCDF(cacheResult.resultFileName)
        count = count + len(df['x'])
        asyncquery.releaseCache(cacheResult.resultFileName)
    else:
        print("Unexpected error... ")
        
    return count

In [ ]:
totalcount = 0

for rowId, gridCell  in gridCellsDf.iterrows():
    count = checkGridCell(gridCell)
    totalcount = count + totalcount
   
    print("%d CumCount=%d MinX=%d MinY=%d QueryCount=%d GridCellCount=%d"%(rowId, totalcount, gridCell['gridCellMinX'], gridCell['gridCellMinY'],count, gridCell['totalPoints']))

print( 'Number of rows checked %d' % (totalcount))

0 CumCount=7 MinX=1700000 MinY=1100000 QueryCount=7 GridCellCount=7
1 CumCount=156 MinX=1300000 MinY=1400000 QueryCount=149 GridCellCount=149
2 CumCount=17665 MinX=1900000 MinY=-300000 QueryCount=17509 GridCellCount=17509
3 CumCount=54559 MinX=1900000 MinY=-400000 QueryCount=36894 GridCellCount=36894
4 CumCount=102854 MinX=1800000 MinY=-500000 QueryCount=48295 GridCellCount=48295
5 CumCount=126399 MinX=1800000 MinY=-700000 QueryCount=23545 GridCellCount=23545
6 CumCount=138917 MinX=1700000 MinY=1000000 QueryCount=12518 GridCellCount=12518
7 CumCount=147679 MinX=1700000 MinY=0 QueryCount=8762 GridCellCount=8762
8 CumCount=165501 MinX=1700000 MinY=-100000 QueryCount=17822 GridCellCount=17822
9 CumCount=196190 MinX=1700000 MinY=-400000 QueryCount=30689 GridCellCount=30689
10 CumCount=221906 MinX=1700000 MinY=-700000 QueryCount=25716 GridCellCount=25716
11 CumCount=235104 MinX=1700000 MinY=-900000 QueryCount=13198 GridCellCount=13198
12 CumCount=260311 MinX=1600000 MinY=1200000 QueryCount=

101 CumCount=3054910 MinX=900000 MinY=-100000 QueryCount=25034 GridCellCount=25034
102 CumCount=3094418 MinX=900000 MinY=-400000 QueryCount=39508 GridCellCount=39508
103 CumCount=3110273 MinX=900000 MinY=-500000 QueryCount=15855 GridCellCount=15855
104 CumCount=3151026 MinX=900000 MinY=-600000 QueryCount=40753 GridCellCount=40753
105 CumCount=3230172 MinX=900000 MinY=-900000 QueryCount=79146 GridCellCount=79146
106 CumCount=3231902 MinX=900000 MinY=-1000000 QueryCount=1730 GridCellCount=1730
107 CumCount=3260726 MinX=900000 MinY=-1200000 QueryCount=28824 GridCellCount=28824
108 CumCount=3386833 MinX=800000 MinY=1500000 QueryCount=126107 GridCellCount=126107
109 CumCount=3405585 MinX=800000 MinY=1300000 QueryCount=18752 GridCellCount=18752
110 CumCount=3436622 MinX=800000 MinY=1200000 QueryCount=31037 GridCellCount=31037
111 CumCount=3477912 MinX=1600000 MinY=-1100000 QueryCount=41290 GridCellCount=41290
112 CumCount=3516371 MinX=800000 MinY=800000 QueryCount=38459 GridCellCount=38459
1

201 CumCount=7198545 MinX=400000 MinY=100000 QueryCount=56096 GridCellCount=56096
202 CumCount=7340991 MinX=200000 MinY=300000 QueryCount=142446 GridCellCount=142446
203 CumCount=7526501 MinX=200000 MinY=100000 QueryCount=185510 GridCellCount=185510
204 CumCount=7617551 MinX=200000 MinY=-300000 QueryCount=91050 GridCellCount=91050
205 CumCount=7633883 MinX=100000 MinY=1600000 QueryCount=16332 GridCellCount=16332
206 CumCount=7657105 MinX=100000 MinY=1500000 QueryCount=23222 GridCellCount=23222
207 CumCount=7725755 MinX=100000 MinY=1300000 QueryCount=68650 GridCellCount=68650
208 CumCount=7823426 MinX=100000 MinY=1200000 QueryCount=97671 GridCellCount=97671
209 CumCount=7846161 MinX=200000 MinY=900000 QueryCount=22735 GridCellCount=22735
210 CumCount=7895427 MinX=100000 MinY=1000000 QueryCount=49266 GridCellCount=49266
211 CumCount=7922119 MinX=100000 MinY=900000 QueryCount=26692 GridCellCount=26692
212 CumCount=7987501 MinX=600000 MinY=-500000 QueryCount=65382 GridCellCount=65382
213 C

300 CumCount=16925406 MinX=-800000 MinY=-600000 QueryCount=55980 GridCellCount=55980
301 CumCount=16937473 MinX=-800000 MinY=-800000 QueryCount=12067 GridCellCount=12067
302 CumCount=16957343 MinX=-900000 MinY=-100000 QueryCount=19870 GridCellCount=19870
303 CumCount=16995687 MinX=800000 MinY=-1200000 QueryCount=38344 GridCellCount=38344
304 CumCount=17092177 MinX=-900000 MinY=-200000 QueryCount=96490 GridCellCount=96490
305 CumCount=17141301 MinX=-900000 MinY=-300000 QueryCount=49124 GridCellCount=49124
306 CumCount=17267374 MinX=1100000 MinY=700000 QueryCount=126073 GridCellCount=126073
307 CumCount=17305732 MinX=100000 MinY=1400000 QueryCount=38358 GridCellCount=38358
308 CumCount=17403953 MinX=-1000000 MinY=0 QueryCount=98221 GridCellCount=98221
309 CumCount=17441873 MinX=1100000 MinY=-100000 QueryCount=37920 GridCellCount=37920
310 CumCount=17488020 MinX=-1000000 MinY=-100000 QueryCount=46147 GridCellCount=46147
311 CumCount=17523891 MinX=-1000000 MinY=-200000 QueryCount=35871 Gri

In [11]:
from os import listdir
import MalardHelpers as m 

cdfDir = '/data/puma1/scratch/v2/malard/tempnetcdfs/'

files = [f for f in listdir(cdfDir)]
sourcecount = 0
for f in files:
    df = m.getDataFrameFromNetCDF(cdfDir + f)
    sourcecount = sourcecount + len(df)

print('Source rows %d'%(sourcecount))

Source rows 83171977


In [ ]:
#MinX=-600000 MinY=300000 QueryCount=64074 GridCellCount=64077

MinX=-600000
MaxX = MinX + 100000
MinY=300000
MaxY=MinY+100000

shards = json_normalize(json.loads(query.getShards(parentDs, dataSet, region, MinX, MaxX, MinY, MaxY, minT, maxT )))


In [ ]:
count = 0

for s in shards['shardName']:
    df = m.getDataFrameFromNetCDF(s)
    time = datetime.datetime.fromtimestamp(df['time'].min())
    startTime = datetime.datetime( time.year, time.month, 1, 0, 0,0  )  
    endTime = startTime +relativedelta(months=+1)-timedelta(seconds=1)
    
    result = asyncquery.executeQuery( parentDs, dataSet, region, MinX, MaxX, MinY, MaxY, startTime, endTime,[],[]   )
    dfRes = m.getDataFrameFromNetCDF( result.resultFileName )
    
    print( 'StartTime %s EndTime %s ShardCount %d QueryCount %d' % (startTime, endTime, len(df['x']), len(dfRes['x'])))
    
    count = count + len(df['x'])
    
print(count)

In [ ]:
maxT = datetime.datetime(2019,12,30,23,59,59)
minT = datetime.datetime(2012,1,1,0,0,0)

result = asyncquery.executeQuery( parentDs, dataSet, region, MinX, MaxX, MinY, MaxY, minT, maxT,[],[]   )

dfRes = m.getDataFrameFromNetCDF( result.resultFileName )

print(len(dfRes['x'])  ) 

dfRes.to_csv("first.csv")

In [15]:

swathDetails = query.getSwathDetails(parentDs, dataSet,region)
